# Imports

In [ ]:
!!pip install litellm
import os
from google.colab import userdata

In [ ]:
from openai import OpenAI
from typing import List, Dict
import sys
import json
import re

# Handle Interactions

This function handles user interactions with the LLM using ChatML, with each messaging expected to have a system, user, or assistant.

In [ ]:
def generate_response(messages: List[Dict]) -> str:
   """Call LLM to get response using Deepseek."""
   # Retrieve Deepseek API key from Colab secrets and set environment variable
   deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
   client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

   response = client.chat.completions.create(
      model="deepseek-reasoner", # Specify the Deepseek model
      messages=messages,
      max_tokens=1024,
      stream=False
   )
   return response.choices[0].message.content

def list_files() -> List[str]:
    """List all files in the current directory."""
    return [f for f in os.listdir('.') if os.path.isfile(f)]

def read_file(file_name: str) -> str:
    """Read the content of a file."""
    with open(file_name, 'r') as f:
        return f.read()

In [ ]:
goals = """
You are a professional career assistant.

Your role is to help job seekers tailor their resumes effectively to match job descriptions and to proofread resumes to be grammatically correct.

Be empathetic, encouraging, and maintain a professional tone.
"""

actions = """
Ensure that the responses are highly relevant to the job seeker's field of interest, emphasizing the skills, experiences, and qualifications that are most pertinent to the specific job description provided.

Please modify the resume to:
- Be ATS-friendly for the given target occupation and job description, including action verbs for accomplishments, highlighting specified skills, and ensuring that the format complies with ATS standards, including a cover letter outline.
- Reformat the new resume into a consistent format using Times New Roman font.
- Ensure that the resume reflects the user's unique qualifications and strengths while emphasizing the skills and experiences that match.
- Adjust the bullet points under each role to emphasize relevant skills and achievements.
- Make sure that experiences are presented in a way that matches the required qualifications.
- Maintain clarity, conciseness, and professionalism throughout.
- Be grammatically correct, be free of typos, and use proper punctuation.
- Keep the new resume in a readable format that is within or under one-page long.
- Use action verbs wherever possible and sensible.
"""

information = """
Be mindful of any items in the memory and make sure that the logic follows in subsequent outputs.
"""

language = """
Respond in this format:

```
Name: <User's Name>
Industry: <Industry>
Occupation: <Desired Occupation>

Here's your new resume (in markdown) based on the job description and your existing resume:
# <Name in all caps>
<Contact information>

<separator>

## Professional Summary
<Professional summary>

<separator>

## Skills
<Skills>

<separator>

## Experience
<Professional experience>

<separator>

## Education
<Education>

<separator>

## <Recognition or Awards or Certificates>
<Recognition, awards, and/or certificates>
"""

In [ ]:
agent_rules = [{
    "role": "system",
    "content": f"""
{goals}

{actions}

{information}

{language}
"""}]

In [ ]:
memory = [
    {"role": "system", "content": "Keep responses within 500 words total."},
]

In [ ]:
print("Hello, I am a resume improvement agent who can help you improve your resume.")

occupation = input("Please enter your desired occupation: ")
industry = input("Please enter the industry you are interested in: ")
job_description = input("Please enter the job description: ")
name = input("Please enter your name: ")
contact_information = input("Please enter your contact information: ")
summary = input("Please enter your professional summary: ")
skills = input("Please enter your skills: ")
experience = input("Please enter your professional experience: ")
education = input("Please enter your education: ")
recognition_awards_certificates = input("Please enter your recognition, awards, and/or certificates: ")

# Add user's new query to memory
memory.append({"role": "user", "content": f"""
I have a resume that I want formatted in Markdown and a job description.
Please adapt my resume to better align with the job requirements while
maintaining a professional tone. Tailor my skills, experiences, and
achievements to highlight the most relevant points for the position.
Ensure that my resume still reflects my unique qualifications and strengths
but emphasizes the skills and experiences that match the job description,
tailored to {occupation} roles in the {industry} industry.

### Here is the job description:
{job_description}

### My name is: {name}.

### Here is my contact information in plaintext:
{contact_information}

### Here is my professional summary in plaintext:
{summary}

### Here are my skills in plaintext:
{skills}

### Here is my professional experience in plaintext:
{experience}

### Here is my education in plaintext:
{education}

### Here are my recognition, awards, and/or certificates in plaintext:
{recognition_awards_certificates}
"""})

# 1. Construct prompt: Combine agent rules and the memory.
prompt = agent_rules + memory

# 2. Generate response from LLM
print("Agent thinking...")
response = generate_response(prompt)
print(f"Agent response: {response}")

# 3. The LLM's response directly adheres to the agent_rules format.

# 4. Update memory with the LLM's response
memory.append(
    {"role": "assistant", "content": response}
)


Hello, I am a resume improvement agent who can help you improve your resume.
Please enter your desired occupation: Associate Data Analyst
Please enter the industry you are interested in: Nonprofit
Please enter the job description: About the Role  The Associate, Data Analytics ensures that Good Reason Houston’s efforts are shaped and sharpened by insights from rigorous data analysis, and that insights can be shared across the community to accelerate and amplify our impact. This role will report to the Director, Research & Data Strategy and work with other functional teams as needed.     The Associate, Data Analytics will serve as a front line data analyst and data manager. Following data documentation and structuring best practices, as well as internal team expectations, the Associate is responsible for data collection, compilation, cleaning, and maintenance. In collaboration with the Managers, Research & Data, the Associate will maintain our database and create data dashboards. Subsequ

# Notebook Agent Results

The following is the markdown output of the above edited resume:

# CHRISTOPHER FORNESA
Sugar Land, TX | (346) 762-3020 | cfornesa@outlook.com | linkedin.com/in/cfornesa

---

## Professional Summary
Creative data and analytics professional with five years of experience transforming complex data into actionable insights to drive social impact. Skilled in quantitative analysis, data management, and compelling storytelling to support data-driven decision-making in mission-driven environments. Passionate about applying ethical, rigorous data practices to improve educational outcomes and community well-being.

---

## Skills
*   **Data Analysis & Research:** Quantitative Analysis, Research Design, Program Evaluation, Data-Driven Decision Making, Statistical Analysis, Data Mining
*   **Technical Tools:** R, Python (Pandas, NumPy), SQL, Power BI, Tableau, Microsoft Excel, Azure, SharePoint
*   **Data Management & Communication:** Data Cleaning & Compilation, Data Visualization, Dashboard Development, Report Writing, Data Storytelling, Cross-Functional Collaboration, Stakeholder Engagement

---

## Experience
**Chevron** — Houston, TX
*Mobile Data SME | May 2024 – July 2025*
*   Developed and managed data solutions to provide clear, actionable insights from mobility data, directly supporting strategic decisions.
*   Optimized data governance and IT architecture by managing a SharePoint platform, ensuring data integrity and accessibility.
*   Collaborated with cross-functional teams to translate complex technical data into understandable formats for diverse audiences.

*IT Analyst | December 2020 – April 2024*
*   Delivered enterprise-level data and technical support, improving workflows and aligning data systems with key organizational goals.
*   Partnered with leadership to analyze processes and implement improvements, enhancing operational efficiency.

*Network BI Analyst | March 2020 – December 2020*
*   Designed and implemented intuitive Power BI dashboards to increase data transparency and enable faster, evidence-based decisions.
*   Analyzed complex network performance data to identify gaps and presented key findings and recommendations to management.

---

## Education
**MS in Data Science** | Boston University | Expected December 2025
Machine Learning, Data Mining, Statistical Analysis, and Ethics in Data Science | GPA 4.0

**BA in Liberal Studies** | University of Houston | Expected December 2025
Combined curriculum in art, business, and political science, developing strong analytical and critical-thinking skills for policy research | GPA 4.0, Summa Cum Laude

---

## Certificates
Google Business Intelligence Certificate, Coursera (2025)
Google Advanced Data Analytics Certificate, Coursera (2025)
Google Data Analytics Certificate, Coursera (2025)

**Cover Letter Outline:**
*   **Opening Paragraph:** Express enthusiasm for the Associate, Data Analytics role at Good Reason Houston, highlighting a shared passion for using data to improve educational outcomes.
*   **Body Paragraph 1:** Connect your data management and analysis experience (e.g., data cleaning at Chevron, dashboard creation with Power BI) directly to the primary responsibilities of data management, research, and visualization.
*   **Body Paragraph 2:** Emphasize your skills in data storytelling and cross-functional collaboration, showcasing your ability to communicate insights and partner with internal teams to support strategic goals.
*   **Closing Paragraph:** Reiterate your commitment to ethical, data-driven public service

## Live Agent Results

Asking the same exact questions to the <a href="https://cfornesa.com/resume-editor-agent">Live Resume Editor Agent</a> yielded the following output.

Name: Christopher Fornesa
Industry: Nonprofit
Occupation: Associate Data Analyst

Here's your new resume (in markdown) based on the job description and your existing resume:
# CHRISTOPHER FORNASA
Sugar Land, TX | (346) 762-3020 | cfornesa@outlook.com | linkedin.com/in/cfornesa
---
## Professional Summary
Analytical and mission-driven data professional with 5+ years of experience transforming complex datasets into actionable insights. Proven expertise in data management, quantitative analysis, and data storytelling to support research and strategic objectives. Passionate about applying rigorous data analysis to improve educational outcomes and drive community impact in a fast-paced nonprofit environment.
---
## Skills
**Data Analysis & Research:** Quantitative Analysis, Research Design, Data Cleaning & Management, Program Evaluation, Statistical Analysis, Data-Driven Decision Making, Working with Large Administrative Datasets
**Technical & Visualization:** Python (Pandas, NumPy), R, SQL, Tableau, Power BI, SPSS, Alteryx, Stata, Microsoft Excel, PowerPoint, Azure, SharePoint
**Project & Stakeholder Management:** Cross-Functional Collaboration, Stakeholder Engagement, Internal Client Management, Process Improvement, Report Writing, Data Storytelling, Project Planning
---
## Experience
**Chevron** — Houston, TX
**Mobile Data Subject Matter Expert** | May 2024 – July 2025
- Developed and maintained data solutions that provided clear, actionable insights from complex mobility data for diverse audiences.
- Managed a SharePoint platform, optimizing IT architecture and implementing data governance best practices for accurate and accessible information.
- Collaborated with cross-functional teams to ensure data integrity and supported data-driven decision-making across the organization.

**IT Analyst** | December 2020 – April 2024
- Delivered enterprise-level technical and data support, translating complex technical information into understandable insights for non-technical stakeholders.
- Partnered with leadership to analyze and improve departmental workflows, aligning data systems with strategic organizational goals.
- Executed recurring and ad-hoc data analysis to identify operational efficiencies and support strategic projects.

**Network Business Intelligence Analyst** | March 2020 – December 2020
- Designed, developed, and implemented intuitive Power BI dashboards to increase data transparency and enable faster, evidence-based decision-making for management.
- Analyzed complex network performance data to identify key trends, gaps, and opportunities, presenting findings clearly to senior leadership.
---
## Education
**Master of Science in Data Science** | Boston University
*Expected December 2025*
Relevant Coursework: Machine Learning, Data Mining, Statistical Analysis, and Ethics in Data Science | GPA 4.0

**Bachelor of Arts in Liberal Studies** | University of Houston
*Expected December 2025*
Concentration in Political Science and Policy | *Summa Cum Laude*, GPA 4.0
---
## Certifications
Google Business Intelligence Certificate, Coursera | 2025
Google Advanced Data Analytics Certificate, Coursera | 2025
Google Data Analytics Certificate, Coursera | 2025